In [3]:
import langchain
import pandas
from sentence_transformers import SentenceTransformer, util
from utils import *
from langchain.text_splitter import TokenTextSplitter


# Reading my RESUME
with open("resume.txt", "r") as file:
    # Read the entire content of the file
    text_to_split = file.read()

text_splitter = TokenTextSplitter(
    encoding_name="gpt2", chunk_size=250, chunk_overlap=10
)
texts = text_splitter.split_text(text_to_split)

/Users/rafaeldavila/Documents/Duke/Sem2/Cloud/miniprojects/7_vectordb/vectordatabase/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
texts = list(map(lambda x: x.replace("\n", " "), texts))

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
# embedding_list = list(map(lambda sentence: list(model.encode(sentence)),texts))

embedding_list = model.encode(texts)

In [5]:
import chromadb

chroma_client = chromadb.Client()

# Create or load a collection
collection_name = "mycollection"

# Get all collections
collections = chroma_client.list_collections()

# Extract names from collection objects
collection_names = [collection.name for collection in collections]

# Check if the collection exists
if collection_name in collection_names:
    # Load the existing collection
    collection = chroma_client.get_collection(collection_name)
    print("Loaded existing collection:", collection_name)
else:
    # Create a new collection and add data to it
    collection = chroma_client.create_collection(name=collection_name)
    print("Created new collection:", collection_name)


collection.add(
    embeddings=embedding_list,
    documents=texts,
    ids=["ID" + str(i) for i in range(len(embedding_list))],
)

Loaded existing collection: mycollection


In [6]:
similar_documents = collection.query(query_texts=["Tell me about sodemca"], n_results=3)

In [8]:
similar_documents

{'ids': [['ID3', 'ID0', 'ID4']],
 'distances': [[1.5084185600280762, 1.5443986654281616, 1.5851678848266602]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [["ulbright Overview:  Rafael Dávila Bugarín, a notable graduate of Universidad Autónoma de Aguascalientes (UAA), has received the prestigious Fulbright-García Robles scholarship for his second master's degree in the USA. He's chosen Duke University in North Carolina for his studies in Data Science.  The Fulbright-García Robles scholarship has benefited around 5,000 scholars from Mexico and the USA over 30 years. It's administered by COMEXUS and is highly respected for its rigorous selection process.  Rafael also works at the Instituto Federal de Telecomunicaciones and values the role of mathematics in his career. He's passionate about inspiring young minds through mathematics clubs for children.  Rafael Dávila Bugarín will begin his studies in the USA this August, returning to Mexico afterward as per Fulbr

In [10]:
context = [" ".join(sublist) for sublist in similar_documents["documents"]]

In [23]:
import ollama

question = "Tell me about Rafael Davila"
content = f"Answer the following question:{question}\
    With this context:{context}.\
    Do not provide answers outside this context."

response = ollama.chat(
    model="llama2",
    messages=[
        {"role": "user", "content": content},
    ],
)
print(response["message"]["content"])

Rafael Davila Bugarín is a Mexican data scientist who has received the Fulbright-García Robles scholarship to pursue his second master's degree in Data Science at Duke University in North Carolina, USA. He works at the Instituto Federal de Telecomunicaciones and values the role of mathematics in his career. Rafael is also passionate about inspiring young minds through mathematics clubs for children.

Rafael's technical skills include Python, PyTorch, Tensorflow, PySpark, R, SQL, Tableau, Power BI, git, emacs, docker, and AWS, among others. He has relevant experience as a research assistant at the Duke Center for Research & Engineering of AI Technology in Education, where he develops AI-based software tools to enhance teaching and learning.

Rafael is also involved in various educational initiatives, including the Mathematics Club and Mathematics Olympiads, which aim to promote mathematics education in Mexico. He is part of the SODEMCA Overview, a summer course that encourages critical 